In [2]:
import pandas as pd

Se lee el archivo csv que contiene los datos

In [3]:
data = pd.read_csv("./Electric_Vehicle_Population_Data.csv")
data.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1C4JJXP66P,Kitsap,Poulsbo,WA,98370.0,2023,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,0.0,23.0,258127145,POINT (-122.64681 47.73689),PUGET SOUND ENERGY INC,5.303509e+10
1,1G1FX6S08K,Snohomish,Lake Stevens,WA,98258.0,2019,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238.0,0.0,44.0,4735426,POINT (-122.06402 48.01497),PUGET SOUND ENERGY INC,5.306105e+10
2,WBY1Z2C58F,King,Seattle,WA,98116.0,2015,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81.0,0.0,34.0,272697666,POINT (-122.41067 47.57894),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,5YJ3E1EBXK,King,Seattle,WA,98178.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,0.0,37.0,477309682,POINT (-122.23825 47.49461),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
4,5YJSA1V24F,Yakima,Selah,WA,98942.0,2015,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,0.0,15.0,258112970,POINT (-120.53145 46.65405),PACIFICORP,5.307700e+10


Se revisa cuantos registros son nulos en cada columna

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223995 entries, 0 to 223994
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         223995 non-null  object 
 1   County                                             223992 non-null  object 
 2   City                                               223992 non-null  object 
 3   State                                              223995 non-null  object 
 4   Postal Code                                        223992 non-null  float64
 5   Model Year                                         223995 non-null  int64  
 6   Make                                               223995 non-null  object 
 7   Model                                              223995 non-null  object 
 8   Electric Vehicle Type                              223995 non-null  object

Se definen las funciones con las que se realizará la transformación de los datos.

In [5]:
def drop_empty_values(data: pd.DataFrame) -> pd.DataFrame:
    new_data = data.dropna()
    return new_data

def drop_col(data: pd.DataFrame, cols: list) -> pd.DataFrame:
    new_data = data.drop(cols, axis=1)
    return new_data

def separate(lista: list) -> str:
    for i in range(lista.count("")):
        lista.remove("")
    return "|".join(lista)

Debido a que la cantidad de registros nulos en cada columna no excede el 10% del total de los datos es posible eliminarlos sin perdida significativa de información

In [6]:
data = drop_empty_values(data)

Ahora se observa que todas las columnas contienen el mismo número de registros

Por otro lado, existen columnas en el conjunto de datos que no aportan información relevante y deben ser eliminadas. Estas columnas son: Postal Code, Base MSRP (La mayoría de sus valores son 0), Legislative District y Vehicle Location 

In [7]:
cols_drop = ["Postal Code","Legislative District","Vehicle Location","Base MSRP"]
data = drop_col(data, cols_drop)

In [ ]:
electric = data["Electric Utility"].str.split("|")
electric = electric.apply(separate)
data = drop_col(data, "Electric Utility")
data["Electric Utility"] = electric
electric = data["Electric Utility"].str.split("|",expand=True)
data = pd.concat([data,electric],axis=1)
data = drop_col(data, "Electric Utility")
data = data.rename(columns={0:"Electric Utility 1", 1: "Electric Utility 2", 2: "Electric Utility 3", 3: "Electric Utility 4"})

,VIN (1-10),County,City,State,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,DOL Vehicle ID,2020 Census Tract,Electric Utility 1,Electric Utility 2,Electric Utility 3,Electric Utility 4
0,1C4JJXP66P,Kitsap,Poulsbo,WA,2023,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,258127145,5.303509e+10,PUGET SOUND ENERGY INC,None,None,None
1,1G1FX6S08K,Snohomish,Lake Stevens,WA,2019,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238.0,4735426,5.306105e+10,PUGET SOUND ENERGY INC,None,None,None
2,WBY1Z2C58F,King,Seattle,WA,2015,BMW,I3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,81.0,272697666,5.303301e+10,CITY OF SEATTLE - (WA),CITY OF TACOMA - (WA),None,None
3,5YJ3E1EBXK,King,Seattle,WA,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,477309682,5.303301e+10,CITY OF SEATTLE - (WA),CITY OF TACOMA - (WA),None,None
4,5YJSA1V24F,Yakima,Selah,WA,2015,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,208.0,258112970,5.307700e+10,PACIFICORP,None,None,None


In [9]:
data.to_csv("Electric_Vehicle_Population_Data_Clean.csv", index=False)